In [33]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np\
import re

# PART ONE

In [90]:
df = pd.read_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\train.csv")
df['Title'] = df['Name'].map(lambda x: re.compile(",(.*?)\.").findall(x)[0])
df['Title'] = df['Title'].map(str.strip) # 匹配的逗号后面有空格，记得去除空格，不然下一步没法替换
df['Title'][df.Title=='Jonkheer'] = 'Master'  #master意为大师
df['Title'][df.Title.isin(['Ms','Mlle'])] = 'Miss'   #多个带判断值时，用isin 比 == 好用。
df['Title'][df.Title.isin(['Mme','Dona', 'Lady', 'the Countess'])] = 'Mrs'
df['Title'][df.Title.isin(['Capt', 'Don', 'Major', 'Col', 'Sir'])] = 'Mr'
df['Title'][df.Title.isin(['Dr','Rev'])] = 'DrAndRev'
df['Title'] = pd.factorize(df.Title)[0]
df['Sex'] = pd.factorize(df.Sex)[0]
df['Familysize'] = df['SibSp'] + df["Parch"]
df['Familysize'][df.Familysize==0] = 0
df['Familysize'][df.Familysize>0] = 1 # 分两类吧，带了亲戚，没带亲戚
#import pylab as pl
##在开头添加pylab的内嵌语句，pylab是 Matplotlib 和Ipython提供的一个模块，提供了类似Matlab的语法。
#%pylab inline
#pl.scatter(df.index,df['Age'])
#pl.scatter(df.index,df['Fare'])
df['Fare'][df.Fare<20] = 0
df['Fare'][df.Fare>=20] = 1
df_tag = df.Survived
df_tag.to_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv',  index=None)
df.drop(['PassengerId', 'Name', 'SibSp' ,'Parch','Ticket', 'Cabin','Embarked'],inplace=True, axis=1)
df['Age'][(df.Age<18)&(df.Age.notnull())] = 0
df['Age'][(df.Age>=18)&(df.Age.notnull())] = 1
#这块是处理Age的缺失值。
y_train = df['Age'][df.Age.notnull()].values
x_train = df[df.Age.notnull()].drop(['Age'],axis=1).values
x_test = df[df.Age.isnull()].drop(['Age'],axis=1).values
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier().fit(x_train,y_train)
df['Age'][df.Age.isnull()] = rfc.predict(x_test)
df.drop(['Survived'],axis=1,inplace=True)
df.to_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None, index=None)
df.head()

,Pclass,Sex,Age,Fare,Title,Familysize
0,3,0,1.0,0.0,0,1
1,1,1,1.0,1.0,1,1
2,3,1,1.0,0.0,2,0
3,1,1,1.0,1.0,1,1
4,3,0,1.0,0.0,0,0


# PART TWO

In [54]:
df = pd.read_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv")
df['Title'] = df['Name'].map(lambda x: re.compile(",(.*?)\.").findall(x)[0])
df['Title'] = df['Title'].map(str.strip)
df['Title'][df.Title=='Jonkheer'] = 'Master'  #master意为大师
df['Title'][df.Title.isin(['Ms','Mlle'])] = 'Miss'   #多个带判断值时，用isin 比 == 好用。
df['Title'][df.Title.isin(['Mme','Dona', 'Lady', 'the Countess'])] = 'Mrs'
df['Title'][df.Title.isin(['Capt', 'Don', 'Major', 'Col', 'Sir'])] = 'Mr'
df['Title'][df.Title.isin(['Dr','Rev'])] = 'DrAndRev'
df['Title'] = pd.factorize(df.Title)[0]
df['Sex'] = pd.factorize(df.Sex)[0]
df['Familysize'] = df['SibSp'] + df["Parch"]
df['Familysize'][df.Familysize==0] = 0
df['Familysize'][df.Familysize>0] = 1
df['Fare'][df.Fare<20] = 0
df['Fare'][df.Fare>=20] = 1
df.drop(['PassengerId', 'Name', 'SibSp' ,'Parch','Ticket', 'Cabin','Embarked'],inplace=True, axis=1)
df['Age'][(df.Age<18)&(df.Age.notnull())] = 0
df['Age'][(df.Age>=18)&(df.Age.notnull())] = 1
df['Fare'][df.Fare.isnull()] = 10# 注意test的fare有个空值，我手改了
y_train = df['Age'][df.Age.notnull()].values
x_train = df[df.Age.notnull()].drop(['Age'],axis=1).values
x_test = df[df.Age.isnull()].drop(['Age'],axis=1).values
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier().fit(x_train,y_train)
df['Age'][df.Age.isnull()] = rfc.predict(x_test)
df.to_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None, index=None)
df.head()

,Pclass,Sex,Age,Fare,Title,Familysize
0,3,0,1.0,0.0,0,0
1,3,1,1.0,0.0,1,1
2,2,0,1.0,0.0,0,0
3,3,0,1.0,0.0,0,0
4,3,1,1.0,0.0,1,1


#  PART THREE

In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [76]:
def sub():
    x_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None).values
    y_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv', header=None).values
    x_test = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None).values
    #集合方法
    clf1 = svm.SVC(probability=True,random_state=7)
    clf2 = RandomForestClassifier(random_state=7)
    clf3 = GradientBoostingClassifier(random_state=7)
    voting_class = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    #保存模型
    joblib.dump(vote,r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\vote_sub7.model')
    #vote = joblib.load(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\0.7835820895522388vote1.model')
    y_test_pred = vote.predict(x_test)
    pre = pd.DataFrame(y_test_pred,index=None,columns=['Survived'])
    pre.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\pre7.csv",index=None)
    df = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv')
    dfID = df.PassengerId
    sub = pd.concat([dfID,pre],axis=1)
    sub.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub7.csv", index=None)
    print ("ok")
    
if __name__ == '__main__':
    sub()

ok


In [77]:
def sub():
    x_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None).values
    y_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv', header=None).values
    x_test = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None).values
    #集合方法
    clf1 = svm.SVC(probability=True,random_state=6)
    clf2 = RandomForestClassifier(random_state=6)
    clf3 = GradientBoostingClassifier(random_state=6)
    voting_class = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    #保存模型
    joblib.dump(vote,r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\vote_sub6.model')
    #vote = joblib.load(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\0.7835820895522388vote1.model')
    y_test_pred = vote.predict(x_test)
    pre = pd.DataFrame(y_test_pred,index=None,columns=['Survived'])
    pre.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\pre6.csv",index=None)
    df = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv')
    dfID = df.PassengerId
    sub = pd.concat([dfID,pre],axis=1)
    sub.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub6.csv", index=None)
    print ("ok")
    
if __name__ == '__main__':
    sub()

ok


In [78]:
def sub():
    x_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None).values
    y_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv', header=None).values
    x_test = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None).values
    #集合方法
    clf1 = svm.SVC(probability=True,random_state=5)
    clf2 = RandomForestClassifier(random_state=5)
    clf3 = GradientBoostingClassifier(random_state=5)
    voting_class = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    #保存模型
    joblib.dump(vote,r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\vote_sub5.model')
    #vote = joblib.load(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\0.7835820895522388vote1.model')
    y_test_pred = vote.predict(x_test)
    pre = pd.DataFrame(y_test_pred,index=None,columns=['Survived'])
    pre.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\pre5.csv",index=None)
    df = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv')
    dfID = df.PassengerId
    sub = pd.concat([dfID,pre],axis=1)
    sub.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub5.csv", index=None)
    print ("ok")
    
if __name__ == '__main__':
    sub()

ok


In [79]:
def sub():
    x_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None).values
    y_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv', header=None).values
    x_test = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None).values
    #集合方法
    clf1 = svm.SVC(probability=True,random_state=4)
    clf2 = RandomForestClassifier(random_state=4)
    clf3 = GradientBoostingClassifier(random_state=4)
    voting_class = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    #保存模型
    joblib.dump(vote,r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\vote_sub4.model')
    #vote = joblib.load(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\0.7835820895522388vote1.model')
    y_test_pred = vote.predict(x_test)
    pre = pd.DataFrame(y_test_pred,index=None,columns=['Survived'])
    pre.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\pre4.csv",index=None)
    df = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv')
    dfID = df.PassengerId
    sub = pd.concat([dfID,pre],axis=1)
    sub.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub4.csv", index=None)
    print ("ok")
    
if __name__ == '__main__':
    sub()

ok


In [80]:
def sub():
    x_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None).values
    y_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv', header=None).values
    x_test = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None).values
    #集合方法
    clf1 = svm.SVC(probability=True,random_state=3)
    clf2 = RandomForestClassifier(random_state=3)
    clf3 = GradientBoostingClassifier(random_state=3)
    voting_class = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    #保存模型
    joblib.dump(vote,r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\vote_sub3.model')
    #vote = joblib.load(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\0.7835820895522388vote1.model')
    y_test_pred = vote.predict(x_test)
    pre = pd.DataFrame(y_test_pred,index=None,columns=['Survived'])
    pre.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\pre3.csv",index=None)
    df = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv')
    dfID = df.PassengerId
    sub = pd.concat([dfID,pre],axis=1)
    sub.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub3.csv", index=None)
    print ("ok")
    
if __name__ == '__main__':
    sub()

ok


In [81]:
def sub():
    x_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None).values
    y_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv', header=None).values
    x_test = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None).values
    #集合方法
    clf1 = svm.SVC(probability=True,random_state=2)
    clf2 = RandomForestClassifier(random_state=2)
    clf3 = GradientBoostingClassifier(random_state=2)
    voting_class = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    #保存模型
    joblib.dump(vote,r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\vote_sub2.model')
    #vote = joblib.load(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\0.7835820895522388vote1.model')
    y_test_pred = vote.predict(x_test)
    pre = pd.DataFrame(y_test_pred,index=None,columns=['Survived'])
    pre.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\pre2.csv",index=None)
    df = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv')
    dfID = df.PassengerId
    sub = pd.concat([dfID,pre],axis=1)
    sub.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub2.csv", index=None)
    print ("ok")
    
if __name__ == '__main__':
    sub()

ok


In [82]:
def sub():
    x_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv', header=None).values
    y_train = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv', header=None).values
    x_test = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_test_feature.csv', header=None).values
    #集合方法
    clf1 = svm.SVC(probability=True,random_state=1)
    clf2 = RandomForestClassifier(random_state=1)
    clf3 = GradientBoostingClassifier(random_state=1)
    voting_class = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('dt', clf3)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    #保存模型
    joblib.dump(vote,r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\vote_sub1.model')
    #vote = joblib.load(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\0.7835820895522388vote1.model')
    y_test_pred = vote.predict(x_test)
    pre = pd.DataFrame(y_test_pred,index=None,columns=['Survived'])
    pre.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\pre1.csv",index=None)
    df = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\test.csv')
    dfID = df.PassengerId
    sub = pd.concat([dfID,pre],axis=1)
    sub.to_csv(r"C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub1.csv", index=None)
    print ("ok")
    
if __name__ == '__main__':
    sub()

ok


# PART FOUR

In [71]:
def fit():
    feature = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\cleaned_train_feature.csv',header=None).values
    target = pd.read_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\cleaned_data\\train_tag.csv',header=None).values

    x_train, x_test, y_train, y_test = train_test_split(feature,target,test_size=0.15,random_state=None)

    # 随机森林
    rfc = RandomForestClassifier(n_estimators=3,random_state=2)
    rfc.fit(x_train,y_train)
    y_train_pred = rfc.predict(x_train)
    y_test_pred = rfc.predict(x_test)
    tree_train = accuracy_score(y_train, y_train_pred)
    tree_test = accuracy_score(y_test, y_test_pred)
    print('rfc/test accuracies %.4f/%.4f' % (tree_train, tree_test))

    # feature_importance = rfc.feature_importances_
    # feature_importance = 100.0 * (feature_importance / feature_importance.max())
    # print(feature_importance)
    # cvscores = cross_validation.cross_val_score(rfc, feature, target, cv=5)
    # print(cvscores.mean())

    # svm
    svmMod = svm.SVC(probability=True,random_state=2)
    svmMod.fit(x_train, y_train)
    y_train_pred = svmMod.predict(x_train)
    y_test_pred = svmMod.predict(x_test)
    tree_train = accuracy_score(y_train, y_train_pred)
    tree_test = accuracy_score(y_test, y_test_pred)
    print('SVM train/test accuracies %.4f/%.4f' % (tree_train, tree_test))


    # 使用gbdt来分类
    gbdt = GradientBoostingClassifier(random_state=2)
    gbdt = gbdt.fit(x_train, y_train)
    y_train_pred = gbdt.predict(x_train)
    y_test_pred = gbdt.predict(x_test)
    tree_train = accuracy_score(y_train, y_train_pred)
    tree_test = accuracy_score(y_test, y_test_pred)
    print('gbdt/test accuracies %.4f/%.4f' % (tree_train, tree_test))

    # 集合方法 随机性的模型都会有个random_state来控制随机种子，所以得到一个好模型后记下这个值用来复现模型。
    voting_class = VotingClassifier(estimators=[('rfc', rfc), ('gbdt', gbdt),('svm',svmMod)], voting='soft',
                                    weights=[1, 1, 1])
    vote = voting_class.fit(x_train, y_train)
    y_train_pred = vote.predict(x_train)
    y_test_pred = vote.predict(x_test)
    vote_train = accuracy_score(y_train, y_train_pred)
    vote_test = accuracy_score(y_test, y_test_pred)
    print('Ensemble Classifier train/test accuracies %.4f/%.4f' % (vote_train, vote_test))
    #保存模型 ,compress=3解决了生成一堆文件的问题
    joblib.dump(vote,'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\' + str(vote_test) + 'vote1.model',compress=3)
    #vote = joblib.load('vote.model')



if __name__ == '__main__':
    fit()

rfc/test accuracies 0.8494/0.7761
SVM train/test accuracies 0.8402/0.7687
gbdt/test accuracies 0.8481/0.7836
Ensemble Classifier train/test accuracies 0.8481/0.7836


# PART FIVE

In [88]:
def ensemble():
    array1 = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub1.csv'), header=None).values
    array2 = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub2.csv'), header=None).values
    array3 = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub3.csv'), header=None).values
    array4 = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub4.csv'), header=None).values
    array5 = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub5.csv'),header=None).values
    array6 = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub6.csv'), header=None).values
    array7 = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub7.csv'), header=None).values
    newarray = pd.read_csv(open('C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\sub1.csv'), header=None).values


    for i in range(1,len(array1)):
        count = 0
        if (array1[i][1] == str(1)): count = count + 1
        if (array2[i][1] == str(1)): count = count + 1
        if (array3[i][1] == str(1)): count = count + 1
        if (array4[i][1] == str(1)): count = count + 1
        if (array5[i][1] == str(1)): count = count + 1
        if (array6[i][1] == str(1)): count = count + 1
        if (array7[i][1] == str(1)): count = count + 1

        print (count)
        if (count > 3): newarray[i][1] = 1 #相当于在投票，票数>3票就认为获救。
        else: newarray[i][1] = 0  #第i行第1列。
    df = pd.DataFrame(newarray,index=None)
    df.to_csv(r'C:\\Users\\13121\\Desktop\\my_notebook\\titanic\\奇奇怪怪\\finalsub.csv',index=None,header=None)


    # print array2[0],array2[1],array2[1][0],type(array2[1][1])
    # ['PassengerId' 'Survived'] ['892' '0'] 这个array2[1][1]读出来是str格式，我在上边用int比一个都没比出来，要转下型


class Main():
    def __init__(self):
        pass


if __name__ == '__main__':
    ensemble()

0
7
0
0
7
0
7
0
7
0
0
0
7
0
7
7
0
0
0
7
0
7
7
0
7
0
7
0
0
0
0
0
0
0
0
0
7
7
0
0
0
0
0
7
7
0
0
0
7
7
0
0
7
7
0
0
0
0
0
7
0
0
0
7
7
7
7
0
0
7
7
0
7
0
7
0
0
7
0
7
7
0
0
0
0
0
7
7
7
7
7
0
7
0
0
0
7
0
7
0
7
0
0
0
7
0
0
0
0
0
0
7
7
7
7
0
0
7
0
7
7
0
7
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
7
0
0
0
0
0
0
0
0
7
0
0
7
0
0
7
7
0
7
7
7
7
0
0
0
0
0
7
7
0
0
0
0
0
7
7
0
7
7
0
0
7
0
7
0
7
0
0
0
0
0
7
0
7
0
7
7
0
7
7
7
0
7
0
0
7
0
7
0
0
0
0
7
0
0
7
0
7
0
7
0
7
0
7
7
0
7
0
0
0
7
0
0
0
0
0
0
7
7
7
7
0
0
0
0
7
0
7
7
7
0
0
0
0
0
0
0
7
0
0
0
7
7
0
0
0
0
7
0
0
0
7
7
0
7
0
0
0
0
7
7
7
7
0
0
0
0
0
0
0
7
0
0
0
0
7
0
0
0
0
0
0
0
7
7
0
7
0
7
0
0
0
7
7
7
0
0
0
0
0
0
0
0
7
0
7
0
0
0
7
0
0
7
0
0
0
0
0
7
0
0
0
7
7
7
0
7
0
7
7
0
0
0
0
0
7
0
0
7
0
7
7
0
7
0
0
0
7
0
0
7
0
0
7
7
0
0
0
0
0
0
7
7
0
7
0
0
0
0
0
7
0
0
0
7
0
7
0
0
7
0
7
5
0
0
0
0
7
7
7
7
7
0
7
0
0
0
